In [74]:
# Find all of the mutuals (transactions in both directions)
with model.query() as select:
    t = Transaction()
    u = Transaction()
    t.to_account == u.from_account
    u.to_account == t.from_account
    z = select(t.to_account.name, u.to_account.name)

z.results

,name,name2
0,Abigail Shaffer,Bella's Café & Espresso
1,Abigail Shaffer,Crystal Clear Jewelers
2,Amy Silva,Crystal Clear Jewelers
3,Angela Lopez,Sundown Brewery
4,Anthony Rodriguez,Sweet Serenity Chocolates
...,...,...
86,The Rustic Loaf Bakery,Zachary Ferrell
87,Tommy Walter,The Rustic Loaf Bakery
88,Zachary Ferrell,The Rustic Loaf Bakery
89,Zachary Hicks,Bella's Café & Espresso


In [71]:
# Find all of the triangles in the graph:
with model.query() as select:
    t = Transaction()
    u = Transaction()
    v = Transaction()
    t.to_account == u.from_account
    u.to_account == v.from_account
    v.to_account == t.from_account
    z = select(t.to_account.name, u.to_account.name, v.to_account.name)

z.results

,name,name2,name3
0,Blossom Florist & Gifts,Deborah Figueroa,Margaret Hawkins DDS
1,Carla Gray,James Ferrell,Shirley Suarez
2,Carol Tucker,Tommy Walter,The Rustic Loaf Bakery
3,Crystal Robinson,Gina Moore,Old Town Bookstore
4,Deborah Figueroa,Margaret Hawkins DDS,Blossom Florist & Gifts
5,Ethan Adams,Whitney Peters,The Rustic Loaf Bakery
6,Gina Moore,Old Town Bookstore,Crystal Robinson
7,James Ferrell,Shirley Suarez,Carla Gray
8,Margaret Hawkins DDS,Blossom Florist & Gifts,Deborah Figueroa
9,Matthew Foster,Matthew Foster,Matthew Foster


In [67]:
# Find the refunded purchases:
with model.query() as select:
    purchase = Transaction()
    user = purchase.from_account
    merchant = purchase.to_account
    user.account_type == "User"
    merchant.account_type == "Merchant"
    refund = Transaction()
    refund.type = "refund"
    refund.from_account == merchant
    refund.to_account == user
    # assume the transaction being refunded is the most recent one:
    purchase_rank = model.aggregates.rank_desc(
        purchase.date, 
        per=[user.name, merchant.name, refund.date]
    )
    purchase_rank == 1
    z = select(user.name, merchant.name, purchase.date, refund.date)

z.results.head(15)

,name,name2,date,date2
0,Abigail Shaffer,Bella's Café & Espresso,2024-01-26,2024-01-30
1,Abigail Shaffer,Crystal Clear Jewelers,2023-12-27,2024-01-01
2,Amy Silva,Crystal Clear Jewelers,2024-01-16,2024-01-21
3,Angela Lopez,Sundown Brewery,2023-11-10,2023-11-16
4,Anthony Rodriguez,Sweet Serenity Chocolates,2024-02-07,2024-02-13
5,Carol Tucker,The Rustic Loaf Bakery,2023-11-26,2023-11-29
6,Cassandra Gaines,Old Town Bookstore,2023-12-21,2023-12-25
7,Christina Walters,Blossom Florist & Gifts,2023-12-19,2023-12-21
8,Christina Walters,Blue Ribbon Butchers,2023-12-11,2023-12-15
9,Connie Lawrence,Blossom Florist & Gifts,2023-12-02,2023-12-10


In [68]:
z.results["name"].value_counts().head(15)

name
Abigail Shaffer      2
Christina Walters    2
Deborah Figueroa     2
Crystal Robinson     2
Zachary Hicks        2
Cassandra Gaines     1
Ryan Munoz           1
Michael Lewis        1
Michele Williams     1
Monica Herrera       1
Patty Perez          1
Philip Cannon        1
Richard Aguirre      1
Sarah Moore          1
Lisa Archer          1
Name: count, dtype: int64

In [69]:
# Find the top 15 accounts by PageRank score
with model.query() as select:
    account = Account()
    pagerank_score = graph.compute.pagerank(account)
    pagerank_rank = model.aggregates.rank_desc(pagerank_score, account)
    z = select(
        model.alias(pagerank_rank, "rank"),
        model.alias(pagerank_score, "score"),
        account.name,
        account.account_type
    )

z.results.head(15)

,rank,score,name,account_type
0,1,0.058808,Crystal Clear Jewelers,Merchant
1,2,0.046670,Blossom Florist & Gifts,Merchant
2,3,0.041101,Old Town Bookstore,Merchant
3,4,0.040441,Bella's Café & Espresso,Merchant
4,5,0.039580,Sundown Brewery,Merchant
5,6,0.037091,Green Thumb Plant Nursery,Merchant
6,7,0.036706,Sweet Serenity Chocolates,Merchant
7,8,0.028815,Blue Ribbon Butchers,Merchant
8,9,0.026250,Abigail Shaffer,User
9,10,0.025896,The Rustic Loaf Bakery,Merchant


In [82]:
# define a notion of friendship between users:
with model.rule():
    t = Transaction()
    t.from_account.set(friend=t.to_account)

# look for instances where friends make a purchase
# from the same merchant:
with model.query() as select:
    influencer = Account()
    merchant = Account(account_type="merchant")
    t1 = Transaction()
    t2 = Transaction()
    t1.from_account == influencer
    t2.from_account == influencer.friend
    t1.to_account == merchant
    t2.to_account == merchant
    z = select(influencer, count)

len(z.results) # zero results - no influencers

0